In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
data_path = "C:/Users/minkyu/Desktop/open/"
base_path = 'C:/Users/minkyu/Desktop/dacon accident prevention'

In [ ]:
train = pd.read_csv(data_path+"train.csv" )

In [ ]:
train.head(1)

In [ ]:
train['인적사고'].value_counts()

In [ ]:
import re

text = """1. **작업 표면 안전 확보**: 모든 작업 플랫폼에 미끄럼 방지 매트 설치 및 정기적인 점검 실시.
2. **안전 교육 강화**: 미끄러짐 위험 인식과 안전한 작업 방법에 대한 정기 교육 실시.
3. **개인 보호 장비 의무화**: 안전화 및 안전벨트 착용을 강제화하며, 특히 미끄러운 환경에서는 추가적인 미끄럼 방지 기능이 있는 장비 사용 권장.
4. **작업 절차 표준화**: 자재 이동 및 해체 작업 시 안전한 절차를 명확히 정의하고 준수를 감독.
5. **정기적 안전 점검**: 해체 작업"""

# 숫자 + 점(.) 형태 제거 (예: "1.", "2." 등)
text = re.sub(r'\d+\.', '', text)

# **내용** 패턴 제거
text = re.sub(r'\*\*.*?\*\*', '', text)

# : 제거
text = re.sub(r':', '', text)

# 마지막 .을 제외한 모든 .을 ,로 변경
text = re.sub(r'\.(?=.*[^.])', ',', text)

# 줄바꿈 제거
text = re.sub(r'\n', ' ', text)

# 2칸 이상 띄어쓰기 제거 (한 칸으로 변환)
text = re.sub(r'\s+', ' ', text)

print(text.strip())


In [ ]:
df = pd.read_csv("C:/Users/minkyu/Desktop/dacon accident prevention/submission-EXAONE-3.5-32B-Instruct.csv")

In [ ]:
df['재발방지대책 및 향후조치계획']

In [ ]:
# 정규식을 적용하는 함수 정의
def clean_text(text):
    text = re.sub(r'\d+\.', '', text)          # 숫자 + 점(.) 제거 (예: "1.", "2.")
    text = re.sub(r'\*\*.*?\*\*', '', text)    # **내용** 제거
    text = re.sub(r':', '', text)              # : 제거
    text = re.sub(r'\.(?=.*[^.])', ',', text)  # 마지막 .을 제외한 모든 .을 ,로 변경
    text = re.sub(r'\n', ' ', text)            # 줄바꿈 제거
    text = re.sub(r'\s+', ' ', text)           # 2칸 이상 띄어쓰기 제거 (한 칸으로 변환)
    return text.strip()

# 데이터프레임에 적용
df['재발방지대책 및 향후조치계획'] = df['재발방지대책 및 향후조치계획'].apply(clean_text)

# 결과 확인
print(df)

In [ ]:
df['재발방지대책 및 향후조치계획']

In [ ]:
base_path = 'C:/Users/minkyu/Desktop/dacon accident prevention'
sample = pd.read_csv(data_path+"sample_submission.csv")

In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 5. 정제된 문장을 리스트로 변환 후 임베딩
test_results = df['재발방지대책 및 향후조치계획'].tolist()
pred_embeddings = embedding.encode(test_results)  # (샘플 개수, 768)

# 6. submission 파일 로드
submission = pd.read_csv(data_path+"sample_submission.csv", encoding='utf-8-sig')

# 7. submission 파일에 결과 저장
submission.iloc[:, 1] = test_results  # 2번째 컬럼에 정제된 문장 저장
submission.iloc[:, 2:] = pred_embeddings  # 3번째 컬럼부터 임베딩 저장

# 8. CSV 저장



In [ ]:
submission.to_csv('submission-EXAONE-3.5-32B-Instruct_정규식적용.csv', index=False, encoding='utf-8-sig')

In [ ]:
import pandas as pd 
import numpy as np 

from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

In [ ]:
data_path = "C:/Users/minkyu/Desktop/open/"
base_path = 'C:/Users/minkyu/Desktop/dacon accident prevention'

In [ ]:
train = pd.read_csv(data_path+"train.csv")
test = pd.read_csv(data_path+"test.csv")
sample = pd.read_csv(data_path+"sample_submission.csv")

In [ ]:
grouped = train.groupby("인적사고")

In [ ]:
res = {}
cosine_res = []
for name, group in tqdm(grouped):
    plan = group["재발방지대책 및 향후조치계획"]
    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)    
    cosine_res += similarity[similarity.mean(axis=1).argmax()].tolist()
    res[name] = plan.iloc[similarity.mean(axis=1).argmax()]

In [ ]:
arr = cosine_res

# 0.1 단위로 구간을 지정
bins = np.arange(0, 1.1, 0.1)  # 0.0 ~ 1.0을 0.1 간격으로 나눔

# 히스토그램 계산
hist, bin_edges = np.histogram(arr, bins=bins)

# 결과 출력
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

In [ ]:
res

In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer

# 모델 로드
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

# 결과를 저장할 딕셔너리
category_cosine_res = {}  # 카테고리별 유사도 저장
res = {}  # 대표 문장 저장

# 그룹화 및 처리
grouped = train.groupby("인적사고")
for name, group in tqdm(grouped):
    plan = group["재발방지대책 및 향후조치계획"]
    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)
    
    # 평균 유사도가 가장 높은 행의 유사도 값 저장
    max_mean_idx = similarity.mean(axis=1).argmax()
    category_cosine_res[name] = similarity[max_mean_idx].tolist()  # 해당 카테고리의 유사도 리스트
    res[name] = plan.iloc[max_mean_idx]  # 대표 문장 저장

# 범위별 카테고리 분포 계산
bins = np.arange(0, 1.1, 0.1)
category_bin_counts = {name: np.histogram(values, bins=bins)[0] for name, values in category_cosine_res.items()}

# 결과 출력
print("범위별 '인적사고' 카테고리 분포:")
for i in range(len(bins) - 1):
    print(f"\nRange {bins[i]:.1f} - {bins[i+1]:.1f}:")
    for category, counts in category_bin_counts.items():
        if counts[i] > 0:  # 해당 범위에 값이 있는 경우만 출력
            print(f"  {category}: {counts[i]}개")

# 전체 히스토그램 재확인
all_cosine_res = []
for values in category_cosine_res.values():
    all_cosine_res.extend(values)
hist, bin_edges = np.histogram(all_cosine_res, bins=bins)
print("\n전체 히스토그램 확인:")
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

In [ ]:
category_bin_counts = {name: np.histogram(values, bins=bins)[0] for name, values in category_cosine_res.items()}

# 결과 출력: 개수와 비율
print("범위별 '인적사고' 카테고리 분포 (개수 및 비율):")
for i in range(len(bins) - 1):
    print(f"\nRange {bins[i]:.1f} - {bins[i+1]:.1f}:")
    for category, counts in category_bin_counts.items():
        total_in_category = sum(counts)  # 해당 카테고리의 전체 유사도 값 개수
        if counts[i] > 0:  # 해당 범위에 값이 있는 경우만 출력
            percentage = (counts[i] / total_in_category) * 100
            print(f"  {category}: {counts[i]}개 ({percentage:.2f}%)")

# 전체 히스토그램 재확인
all_cosine_res = []
for values in category_cosine_res.values():
    all_cosine_res.extend(values)
hist, bin_edges = np.histogram(all_cosine_res, bins=bins)
print("\n전체 히스토그램 확인:")
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer

# 모델 로드
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

# 결과를 저장할 딕셔너리
category_cosine_res = {}  # 카테고리별 유사도 값 저장
category_sentences = {}   # 카테고리별 문장과 유사도 저장
res = {}                  # 대표 문장 저장

# 그룹화 및 처리
grouped = train.groupby("인적사고")
for name, group in tqdm(grouped):
    plan = group["재발방지대책 및 향후조치계획"]
    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)
    
    # 평균 유사도가 가장 높은 행 선택
    max_mean_idx = similarity.mean(axis=1).argmax()
    cosine_values = similarity[max_mean_idx].tolist()  # 해당 행의 유사도 값들
    sentences = plan.tolist()                          # 모든 문장 리스트
    
    # 카테고리별 유사도와 문장 저장
    category_cosine_res[name] = cosine_values
    category_sentences[name] = list(zip(cosine_values, sentences))  # (유사도, 문장) 쌍으로 저장
    res[name] = plan.iloc[max_mean_idx]  # 대표 문장 저장

# 범위 설정
bins = np.arange(0, 1.1, 0.1)

# 특정 범위(0.6~1.0)만 추출
target_ranges = [(0.6, 0.7), (0.7, 0.8), (0.8, 0.9), (0.9, 1.0)]

# 범위별 문장 출력
print("범위별 '인적사고' 카테고리 문장 (최대 10개씩):")
for lower, upper in target_ranges:
    print(f"\nRange {lower:.1f} - {upper:.1f}:")
    for category, sentence_pairs in category_sentences.items():
        # 해당 범위에 속하는 (유사도, 문장) 쌍 필터링
        filtered_sentences = [(sim, sent) for sim, sent in sentence_pairs if lower <= sim < upper]
        if filtered_sentences:
            print(f"  {category}:")
            # 최대 10개 출력
            for sim, sent in filtered_sentences[:10]:
                print(f"    - {sent} (유사도: {sim:.3f})")
            if len(filtered_sentences) > 10:
                print(f"      ... (총 {len(filtered_sentences)}개)")

# 전체 히스토그램 재확인
all_cosine_res = []
for values in category_cosine_res.values():
    all_cosine_res.extend(values)
hist, bin_edges = np.histogram(all_cosine_res, bins=bins)
print("\n전체 히스토그램 확인:")
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

In [ ]:
# 텍스트 파일로 저장
with open('output.txt', 'w', encoding='utf-8') as f:
    # 범위별 문장 출력
    f.write("범위별 '인적사고' 카테고리 문장 (최대 10개씩):\n")
    for lower, upper in target_ranges:
        f.write(f"\nRange {lower:.1f} - {upper:.1f}:\n")
        for category, sentence_pairs in category_sentences.items():
            # 해당 범위에 속하는 (유사도, 문장) 쌍 필터링
            filtered_sentences = [(sim, sent) for sim, sent in sentence_pairs if lower <= sim < upper]
            if filtered_sentences:
                f.write(f"  {category}:\n")
                # 최대 10개 출력
                for sim, sent in filtered_sentences[:10]:
                    f.write(f"    - {sent} (유사도: {sim:.3f})\n")
                if len(filtered_sentences) > 10:
                    f.write(f"      ... (총 {len(filtered_sentences)}개)\n")

    # 전체 히스토그램 재확인
    all_cosine_res = []
    for values in category_cosine_res.values():
        all_cosine_res.extend(values)
    hist, bin_edges = np.histogram(all_cosine_res, bins=bins)
    f.write("\n전체 히스토그램 확인:\n")
    for i in range(len(hist)):
        f.write(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개\n")

print("결과가 'output.txt' 파일에 저장되었습니다.")

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

In [ ]:
# 문장 예시
preds = [
    "작업 전 작업자 안전교육 및 안전관리 철저에 대한 재발 방지 대책과 향후 조치 계획",
]

gts = [
    "위험성 평가 및 교육을 통해 작업장 내 위험요인과 안전수칙을 근로자에게 전파하고, 근로자 안전교육을 강화하며, 본 사고와 관련된 유사 피해 발생 방지를 위한 안전교육을 실시할 계획.",
]

In [ ]:
# 샘플에 대한 Cosine Similarity 산식
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

In [ ]:
sample_scores = []
for pred, gt in zip(preds, gts):
    # 생성된 답변 내용을 768 Embedding Vector로 변환
    pred_embed = model.encode(pred)
    gt_embed = model.encode(gt)
    
    sample_score = cosine_similarity(gt_embed, pred_embed)
    # Cosine Similarity Score가 0보다 작으면 0으로 간주
    sample_score = max(sample_score, 0)
    print('예측 : ', pred)
    print('정답 : ', gt)
    print('Cosine Similarity Score : ', sample_score)
    print('-'*20)
    sample_scores.append(sample_score)
print('전체 샘플의 Cosine Similarity Score 평균 : ', np.mean(sample_scores))